# Richardson Extrapolation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import sympy
import scipy
from collections.abc import Callable

In [2]:
def Richardson_Extrapolation(f: Callable[[float], float],
                             x: float,
                             h: float,
                             n: int = 9,
                             rtol: float = 1e-10
                            ) -> tuple[float, pd.DataFrame]:
    N = n + 1
    D = np.full(shape=(N,N), fill_value=np.nan, dtype=np.float64)
    D[0, 0] = 0.5 * (f(x + h) - f(x - h)) / h
    for i in range(1, N, 1):
        h = 0.5 * h
        D[i, 0] = 0.5 * (f(x + h) - f(x - h)) / h
        I = i + 1
        p = 1
        for j in range(1, I, 1):
            p = 4*p
            D[i, j] = D[i, j-1] + (D[i, j-1] - D[i-1, j-1]) / (p-1)
        if abs(D[i,i] - D[i-1, i-1]) < rtol:
            break
    d = D[i,i]
    columns = [f'O(h^{2*(k+1)})' for k in range(0, I, 1)]
    D = pd.DataFrame(data=D[:I, :I], columns=columns)
    return (d, D)

In [3]:
def f(x): return np.log(1+x)
d, D = Richardson_Extrapolation(f=f, x=1, h=0.1)
print(f"d = {d:.10f}")
D.fillna(value='', inplace=True)
print(D)

d = 0.5000000000
     O(h^2) O(h^4) O(h^6) O(h^8)
0  0.500417                     
1  0.500104    0.5              
2  0.500026    0.5    0.5       
3  0.500007    0.5    0.5    0.5
